In [1]:
#convolutional NN
import os
import cv2
import numpy as np
from tqdm import tqdm

REBUILD_DATA = False

class DogsVsCats():
    IMG_SIZE = 50
    CATS = "datasets/kaggle_cats_and_dogs/PetImages/Cat"
    DOGS = "datasets/kaggle_cats_and_dogs/PetImages/Dog"
    LABELS = {CATS: 0 , DOGS: 1}
    training_data = []
    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label,f)
                    img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE,self.IMG_SIZE))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])

                    if label == self.CATS:
                        self.catcount += 1
                    else:
                        self.dogcount += 1
                except Exception as e:
                    pass
                    #print(str(e))

        np.random.shuffle(self.training_data)
        np.save("training_data.npy",self.training_data)
        print("Cats:", self.catcount)
        print("Dogs:", self.dogcount)



if REBUILD_DATA:
    dogvcats = DogsVsCats()
    dogvcats.make_training_data()

In [2]:
training_data = np.load("training_data.npy",allow_pickle=True)
print(len(training_data))

24946


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,5) #(input size,ouptuts,kernel size)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv3 = nn.Conv2d(64,128,5)


        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)


        self.fc1 = nn.Linear(self._to_linear,512)
        self.fc2 = nn.Linear(512,2)

    def convs(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1,self._to_linear)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.softmax(x,dim=1)

net = Net()

In [5]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(),lr = 0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0 # scale it down

y = torch.Tensor([i[1] for i in training_data])

validation_percent = 0.1
validation_size = int(len(X)*validation_percent)
print(validation_size)

2494


In [6]:
train_X = X[:-validation_size]
train_y = y[:-validation_size]

test_X = X[-validation_size:]
test_y = y[-validation_size:]

print(len(train_X))
print(len(test_X))

22452
2494


In [7]:
batch_size = 100
epochs = 1

for epoch in range(epochs):
    for i in tqdm(range(0,len(train_X),batch_size)):
        batch_X = train_X[i:i+batch_size].view(-1,1,50,50)
        batch_y = train_y[i:i+batch_size]
        net.zero_grad()
        
        outputs = net(batch_X)
        loss = loss_function(outputs,batch_y)
        loss.backward()
        optimizer.step()

print(loss)

100%|██████████| 225/225 [02:08<00:00,  1.76it/s]tensor(0.2500, grad_fn=<MseLossBackward>)



In [12]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1,1,50,50))[0]
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct += 1
        total += 1

print("Accuracy:", round(correct/total,3))

100%|██████████| 2494/2494 [00:10<00:00, 238.27it/s]Accuracy: 0.491

